<a href="https://colab.research.google.com/github/riamary18/Learning-NLP/blob/main/NLP_with_Disaster_Tweetsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries and packages.

In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction, linear_model, model_selection

## Importing and Cleaning Dataset

In [3]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [4]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
train_df[train_df['target']==0]['text']

15                                         What's up man?
16                                          I love fruits
17                                       Summer is lovely
18                                      My car is so fast
19                           What a goooooooaaaaaal!!!!!!
                              ...                        
7581    @engineshed Great atmosphere at the British Li...
7582    Cramer: Iger's 3 words that wrecked Disney's s...
7584    These boxes are ready to explode! Exploding Ki...
7587                                   Sirens everywhere!
7593    I just heard a really loud bang and everyone i...
Name: text, Length: 4342, dtype: object

In [6]:
train_df[train_df['target']==1]['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 3271, dtype: object

In [27]:
train_df,test_df=train_df.dropna(),test_df.dropna()

### Building Vectors
The words contained in each tweet can tell us if it is a tweet about a real disaster or not. This assumption is a linear connection. So we can use `LinearModel`.
We can use sklearn's `CountVectorizer` to count the words in each tweet and turn them into data our model can process.


In [56]:
combined_data=train_df.append(test_df)

<ipython-input-56-de2778bed718>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data=train_df.append(test_df)


In [57]:
combined_data

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1.0
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0.0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1.0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0.0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0.0
...,...,...,...,...,...
3246,10804,wrecked,Love Reiss,@yakubOObs think he deactivated because his no...,NaN
3247,10806,wrecked,Seattle Washington,RT CNBC '3 words from Disney CEO Bob Iger wrec...,NaN
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...,NaN
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...,NaN


In [69]:
combined_vectors=count_vec.fit_transform(combined_data['text'])

In [70]:
example_train_vectors=count_vec.fit_transform(train_df['text'][0:5])

In [71]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 42)
[[0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0]]


There are 42 unique words (tokens) in the first 5 tweets.
All the non-zero counts are the tokens that DO exist in the first tweet.

In [72]:
train_vectors=count_vec.transform(train_df['text'])
test_vectors=count_vec.transform(test_df['text'])

In [73]:
len(train_vectors.todense()),len(test_vectors.todense())

(5080, 2158)

After testing out a few models, we see that the model `PassiveAggresiveClassifier()` gives the best scores.

In [81]:
clf=linear_model.PassiveAggressiveClassifier()

We'll now test our model and see how well it does on the training data. We can use `cross-validation` for this purpose (train a portion of known data and then validate using the rest).

In [82]:
#The cv parameter represents the number of folds the dataset is split into for cross-validation
#F1 score is usually used in binary classification to assess a model's performance (precision and recall into a single value)
scores=model_selection.cross_val_score(clf,train_vectors,train_df['target'],cv=3,scoring='f1')
scores

array([0.55889724, 0.42834395, 0.59514399])

### Making Predictions on Training Set

In [62]:
clf.fit(train_vectors,train_df['target'])

PassiveAggressiveClassifier()

In [66]:
clf.predict(test_vectors)

array([0, 0, 0, ..., 0, 0, 0])